<a href="https://colab.research.google.com/github/prabjotbali/Disease-Detection-YOLOv5/blob/main/Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="mU31sq0TNh4S1Drak4hl")
project = rf.workspace("apple-oimpi").project("disease-detection-sztru")
dataset = project.version(1).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Disease-Detection-1 in yolov5pytorch:: 100%|██████████| 774/774 [00:00<00:00, 1072.38it/s]


In [ ]:
model = !python /content/yolov5/train.py --img 320 --batch -1 --epochs 100 --data {dataset.location}/data.yaml --weights /content/yolov5/weights/yolov5x.pt --name custom

In [ ]:
%cat {dataset.location}/data.yaml

names:
- Apple
nc: 1
roboflow:
  license: CC BY 4.0
  project: disease-detection-sztru
  url: https://universe.roboflow.com/project/disease-detection-sztru/dataset/1
  version: 1
  workspace: project
test: ../test/images
train: Disease-Detection-1/train/images
val: Disease-Detection-1/valid/images


In [ ]:
!python /content/yolov5/val.py --weights /content/yolov5/runs/train/custom/weights/best.pt --data {dataset.location}/data.yaml --img 640  --name custom

val: data=/content/yolov5/Disease-Detection-1/data.yaml, weights=['/content/yolov5/runs/train/custom/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=custom, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
val: Scanning /content/yolov5/Disease-Detection-1/valid/labels.cache... 76 images, 0 backgrounds, 0 corrupt: 100% 76/76 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 3/3 [00:09<00:00,  3.03s/it]
                   all         76        630      0.825      0.868      0.917      0.686
Speed: 0.3ms pre-process, 59.5ms inference, 5.2ms NMS per ima

In [ ]:
import os

if os.path.exists(os.path.join(dataset.location, "test")):
  split_path = os.path.join(dataset.location, "test", "images")
else:
  split_path = os.path.join(dataset.location, "valid", "images")
example_image_name = os.listdir(split_path)[0]
example_image_path = os.path.join(split_path, example_image_name)

In [ ]:
!python /content/yolov5/detect.py --img 640 --weights /content/yolov5/runs/train/custom/weights/best.pt --source /content/yolov5/Disease-Detection-1/test/images

detect: weights=['/content/yolov5/runs/train/custom/weights/best.pt'], source=/content/yolov5/Disease-Detection-1/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
image 1/38 /content/yolov5/Disease-Detection-1/test/images/IMG_0185_jpg.rf.e0d81820128c672977a4b61b4d696731.jpg: 640x640 23 Apples, 92.0ms
image 2/38 /content/yolov5/Disease-Detection-1/test/images/IMG_0187_jpg.rf.2c66bd09be797aad16666c1292408e8f.jpg: 640x640 7 Apples, 65.1ms
image 3/38 /content/

In [1]:
import joblib

In [ ]:
model

NameError: ignored

In [2]:
joblib.dump(model,"/content/drive/MyDrive/Saved Model/Segmentation_joblib.pkl")

NameError: ignored